<a href="https://colab.research.google.com/github/KhaSab5292/DS-Unit-2-Applied-Modeling/blob/master/Copy_of_assignment_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - _**[A Kaggle Master Explains Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/)**_
  - [_An Introduction to Statistical Learning_](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) Chapter 8
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
  - _**[Boosting](https://www.youtube.com/watch?v=GM3CDQfQ4sw) (2.5 minute video)**_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%%capture
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


DATA_PATH = '/content/drive/My Drive/Lambda/Unit 2 DS9/Unit 2 Build/Police_Department_Incidents_-_Previous_Year__2016_.csv'
!pip install category_encoders==2.*

In [3]:
df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()

(150500, 13)


,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


# **Continue to clean and explore your data. Make exploratory visualizations.**

In [4]:
df.describe()

,IncidntNum,X,Y,PdId
count,1.505000e+05,150500.000000,150500.000000,1.505000e+05
mean,1.616440e+08,-122.423599,37.768921,1.616440e+13
std,5.535976e+06,0.026210,0.023637,5.535976e+11
min,1.135121e+07,-122.513642,37.707922,1.135121e+12
25%,1.603283e+08,-122.434036,37.756486,1.603283e+13
50%,1.606541e+08,-122.416903,37.775421,1.606541e+13
75%,1.609764e+08,-122.406605,37.785063,1.609764e+13
max,9.910090e+08,-122.365565,37.819975,9.910090e+13


In [5]:
df.describe(exclude = 'number')

,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,Location
count,150500,150500,150500,150500,150500,150499,150500,150500,150500
unique,39,726,7,366,1439,10,14,16130,19386
top,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Friday,01/01/2016 12:00:00 AM,12:00,SOUTHERN,NONE,800 Block of BRYANT ST,"(37.775420706711, -122.403404791479)"
freq,40409,17741,23371,558,3842,28445,107780,3561,3536


In [0]:
exluded_columns = ['IncidntNum', 'Descript', 'Date', 'Time', 'PdId', 'Address', 'Location']

In [7]:
df['Resolution'].value_counts()

NONE                                      107780
ARREST, BOOKED                             39416
UNFOUNDED                                   1608
JUVENILE BOOKED                             1056
EXCEPTIONAL CLEARANCE                        371
ARREST, CITED                                144
CLEARED-CONTACT JUVENILE FOR MORE INFO        58
NOT PROSECUTED                                22
LOCATED                                       20
PSYCHOPATHIC CASE                             17
JUVENILE CITED                                 3
JUVENILE DIVERTED                              2
COMPLAINANT REFUSES TO PROSECUTE               2
PROSECUTED BY OUTSIDE AGENCY                   1
Name: Resolution, dtype: int64

In [8]:
df['Category'].value_counts()
#df['PdDistrict'].value_counts()

LARCENY/THEFT                  40409
OTHER OFFENSES                 19599
NON-CRIMINAL                   17866
ASSAULT                        13577
VANDALISM                       8589
VEHICLE THEFT                   6419
WARRANTS                        5914
BURGLARY                        5802
SUSPICIOUS OCC                  5782
MISSING PERSON                  4338
DRUG/NARCOTIC                   4243
ROBBERY                         3299
FRAUD                           2635
SECONDARY CODES                 1841
TRESPASS                        1812
WEAPON LAWS                     1658
SEX OFFENSES, FORCIBLE           940
STOLEN PROPERTY                  882
RECOVERED VEHICLE                736
DISORDERLY CONDUCT               658
PROSTITUTION                     641
FORGERY/COUNTERFEITING           619
DRUNKENNESS                      465
DRIVING UNDER THE INFLUENCE      378
ARSON                            286
KIDNAPPING                       257
EMBEZZLEMENT                     168
L

In [0]:
def clean(X):
  X = X.copy()
  X = X.drop(columns = exluded_columns)
  
  X['Resolution'] = X['Resolution'].replace(['ARREST, BOOKED', 'ARREST, CITED'], 
                                            'ARRESTED')
  X['Resolution'] = X['Resolution'].replace(['JUVENILE BOOKED', 'JUVENILE CITED', 'JUVENILE CITED', 'JUVENILE DIVERTED'], 
                                            'JUVENILE')
  X['Resolution'] = X['Resolution'].replace('COMPLAINANT REFUSES TO PROSECUTE', 'NOT PROSECUTED')
  X['Resolution'] = X['Resolution'].replace(['CLEARED-CONTACT JUVENILE FOR MORE INFO', 'EXCEPTIONAL CLEARANCE'],
                                            'CLEARED')
  
  X['Category'] = X['Category'].replace(['SEX OFFENSES, NON FORCIBLE', 'SEX OFFENSES, FORCIBLE'], 
                                        'SEX OFFENSES')
  X['Category'] = X['Category'].replace(['LARCENY/THEFT', 'ROBBERY', 'BURGLARY', 'STOLEN PROPERTY'], 
                                        'THEFT')
  X['Category'] = X['Category'].replace('FORGERY/COUNTERFEITING', 'FRAUD')

  return X

In [10]:
dfclean = clean(df)
dfclean['Category'].value_counts()

THEFT                          50392
OTHER OFFENSES                 19599
NON-CRIMINAL                   17866
ASSAULT                        13577
VANDALISM                       8589
VEHICLE THEFT                   6419
WARRANTS                        5914
SUSPICIOUS OCC                  5782
MISSING PERSON                  4338
DRUG/NARCOTIC                   4243
FRAUD                           3254
SECONDARY CODES                 1841
TRESPASS                        1812
WEAPON LAWS                     1658
SEX OFFENSES                     980
RECOVERED VEHICLE                736
DISORDERLY CONDUCT               658
PROSTITUTION                     641
DRUNKENNESS                      465
DRIVING UNDER THE INFLUENCE      378
ARSON                            286
KIDNAPPING                       257
EMBEZZLEMENT                     168
LIQUOR LAWS                      156
RUNAWAY                          140
SUICIDE                           69
BRIBERY                           66
E

# **Fit a model. Does it beat your baseline?**

In [11]:
dfclean['Resolution'].value_counts(normalize = True)

NONE                            0.716146
ARRESTED                        0.262857
UNFOUNDED                       0.010684
JUVENILE                        0.007050
CLEARED                         0.002850
NOT PROSECUTED                  0.000159
LOCATED                         0.000133
PSYCHOPATHIC CASE               0.000113
PROSECUTED BY OUTSIDE AGENCY    0.000007
Name: Resolution, dtype: float64

In [27]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

target = 'Resolution'
guess = dfclean[target].mode()
y_pred = [guess] * len(dfclean)
base_auc = accuracy_score(dfclean[target], y_pred)
base_f1 = f1_score(dfclean[target], y_pred, average = 'weighted')
print('Baseline accuracy score:', base_auc)
print('Baseline f1 score:', base_f1)

Baseline accuracy score: 0.7161461794019933
Baseline f1 score: 0.597694248226319


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
print(classification_report(dfclean[target], y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                              precision    recall  f1-score   support

                    ARRESTED       0.00      0.00      0.00     39560
                     CLEARED       0.00      0.00      0.00       429
                    JUVENILE       0.00      0.00      0.00      1061
                     LOCATED       0.00      0.00      0.00        20
                        NONE       0.72      1.00      0.83    107780
              NOT PROSECUTED       0.00      0.00      0.00        24
PROSECUTED BY OUTSIDE AGENCY       0.00      0.00      0.00         1
           PSYCHOPATHIC CASE       0.00      0.00      0.00        17
                   UNFOUNDED       0.00      0.00      0.00      1608

                    accuracy                           0.72    150500
                   macro avg       0.08      0.11      0.09    150500
                weighted avg       0.51      0.72      0.60    150500



In [16]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dfclean, train_size = 0.8, test_size = 0.2,  
                               random_state = 42)

dfclean.shape, train.shape, test.shape

((150500, 6), (120400, 6), (30100, 6))

In [17]:
train, val = train_test_split(train, train_size = 0.75, test_size = 0.25,  
                               random_state = 42)

train.shape, val.shape, test.shape

((90300, 6), (30100, 6), (30100, 6))

In [0]:
# Arrange data into X features matrix and y target vector
target = 'Resolution'
X_train = train.drop(columns = target)
y_train = train[target]
X_val = val.drop(columns = target)
y_val = val[target]

In [35]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy = 'median'), 
    RandomForestClassifier(n_estimators = 100, random_state = 42, n_jobs = -1)
)

# Fit on train
pipeline.fit(X_train, y_train)
# score on val
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.8323588039867109


In [36]:
# f1 score on val
y_pred = pipeline.predict(X_val)
val_f1 = f1_score(y_val, y_pred, average = 'weighted')
print('Validation f1 score:', val_f1)


Validation f1 score: 0.8257955915983274


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                   precision    recall  f1-score   support

         ARRESTED       0.72      0.69      0.70      7838
          CLEARED       0.25      0.07      0.10        92
         JUVENILE       0.38      0.09      0.15       222
          LOCATED       0.33      0.20      0.25         5
             NONE       0.87      0.91      0.89     21606
   NOT PROSECUTED       0.00      0.00      0.00         6
PSYCHOPATHIC CASE       0.00      0.00      0.00         4
        UNFOUNDED       0.57      0.18      0.28       327

         accuracy                           0.83     30100
        macro avg       0.39      0.27      0.30     30100
     weighted avg       0.82      0.83      0.83     30100



# **Try xgboost.**

# **Get your model's permutation importances.**